In [3]:
from sklearn.datasets import fetch_openml

In [9]:
minst = fetch_openml('mnist_784', as_frame=False,parser="auto")

In [11]:
property_names = [prop for prop in dir(minst) if not callable(getattr(minst, prop)) and not prop.startswith("__")]

for name in property_names:
    print(name)

DESCR
categories
data
details
feature_names
frame
target
target_names
url
